In [1]:
import os
import yaml
import tifffile
import numpy as np

import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))
from src import measures
from src.separator import Separator
from src.cleaner import DoubleStepCleaner

In [2]:
fpath = "/mnt/HD-LSDF/Medaka/201912_beamtime_medaka/"
db_path = "/home/ws/er5241/Repos/measuring-repo/artifacts/debug.json"

# fls = os.listdir(fpath)
# id = np.random.choice(fls)

### brain

In [6]:
id = 'Medaka_1115_20-1'
img_path = fpath + id + "/scaled_0.5_8bit_cropped_slices.tif"
img = tifffile.imread(img_path)

main_organ = 'brain'
cfg_path = f"/home/ws/er5241/Repos/measuring-repo/measurement_configs/measurement/{main_organ}.yaml"
msk_path = fpath + id + f"/{main_organ}_scaled_0.5_8bit_cropped_slices.tif"
msk = tifffile.imread(msk_path)

with open(cfg_path, "r") as stream:
    configs = yaml.safe_load(stream)

cleaning_config = configs['cleaning']
measuring_config = configs['measures']
centering_config = configs.get('centering', [])

cleaner = DoubleStepCleaner(**cleaning_config)
msk_clean, roi = cleaner(msk)
img_clean = img[roi]

In [7]:
centers = {}
for centering in centering_config:
    centers[centering['name']] = Separator((msk_clean == centering['label_id']), centering['function'], centering['count'])

In [ ]:
for lbl in measuring_config:
  lbl_mask = (msk_clean == lbl['id'])
  print(lbl['id'], lbl['name'])
  for measure in lbl['measures']:
    if 'eccentricity' not in measure['function']:  # takes too long
      mf = getattr(measures, measure['function'])
      cc = centers[lbl['center']] if ('center' in lbl.keys()) else None
      measurement = mf(lbl_mask, img_clean, cc)
      print(measure['function'],measurement)

### heartkidney

In [9]:
id = 'Medaka_1129_25-1'
img_path = fpath + id + "/scaled_0.5_8bit_cropped_slices.tif"
img = tifffile.imread(img_path)

main_organ = 'heartkidney'
cfg_path = f"/home/ws/er5241/Repos/measuring-repo/measurement_configs/measurement/{main_organ}.yaml"

with open(cfg_path, "r") as stream:
    configs = yaml.safe_load(stream)

cleaning_config = configs['cleaning']
measuring_config = configs['measures']
centering_config = configs.get('centering', [])

msk_path = fpath + id + f"/{main_organ}_scaled_0.5_8bit_cropped_slices.tif"
msk = tifffile.imread(msk_path)
cleaner = DoubleStepCleaner(**cleaning_config)

msk_clean, roi = cleaner(msk)
img_clean = img[roi]

In [10]:
centers = {}
for centering in centering_config:
    centers[centering['name']] = Separator((msk_clean == centering['label_id']), centering['function'], centering['count'])

In [ ]:
for lbl in measuring_config:
  lbl_mask = (msk_clean == lbl['id'])
  print(lbl['id'], lbl['name'])
  for measure in lbl['measures']:
    if 'eccentricity' not in measure['function']:  # takes too long
      mf = getattr(measures, measure['function'])
      cc = centers[lbl['center']] if ('center' in lbl.keys()) else None
      measurement = mf(lbl_mask, img_clean, cc)
      print(measure['function'],measurement)

2 atrium
volume [312297]
surface_area [113895]
color_mean [117.87404938247886]
color_mean_dilated [86.76456396245538]
color_mean_eroded [130.27783755583815]
color_median [120.0]
color_median_dilated [79.0]
color_median_eroded [133.0]
color_std [24.1461341750545]
color_std_dilated [32.53474937276524]
color_std_eroded [20.863448809822167]
color_perc_1 [50.0]
color_perc_1_dilated [37.0]
color_perc_1_eroded [55.0]
color_perc_99 [162.0]
color_perc_99_dilated [160.0]
color_perc_99_eroded [167.0]
convex_volume [593790.3333333333]
radius_minimal_sphere [81.1001415284453]
4 left_kidney
volume [724539]
surface_area [115294]
color_mean [119.95255879945731]
color_mean_dilated [75.15645320749333]
color_mean_eroded [118.79294249168808]
color_median [120.0]
color_median_dilated [67.0]
color_median_eroded [118.0]
color_std [21.833129345456133]
color_std_dilated [32.51693567817804]
color_std_eroded [19.50704670465549]
color_perc_1 [65.0]
color_perc_1_dilated [24.0]
color_perc_1_eroded [72.0]
color_perc

### liver

In [20]:
id = 'Medaka_1115_20-1'
img_path = fpath + id + "/scaled_0.5_8bit_cropped_slices.tif"
img = tifffile.imread(img_path)

main_organ = 'liver'
cfg_path = f"/home/ws/er5241/Repos/measuring-repo/measurement_configs/measurement/{main_organ}.yaml"

with open(cfg_path, "r") as stream:
    configs = yaml.safe_load(stream)

cleaning_config = configs['cleaning']
measuring_config = configs['measures']
centering_config = configs.get('centering', [])

msk_path = fpath + id + f"/{main_organ}_scaled_0.5_8bit_cropped_slices.tif"
msk = tifffile.imread(msk_path)
cleaner = DoubleStepCleaner(**cleaning_config)

msk_clean, roi = cleaner(msk)
img_clean = img[roi]

In [21]:
centers = {}
for centering in centering_config:
    centers[centering['name']] = Separator((msk_clean == centering['label_id']), centering['function'], centering['count'])

In [6]:
for lbl in measuring_config:
  lbl_mask = (msk_clean == lbl['id'])
  print(lbl['id'], lbl['name'])
  for measure in lbl['measures']:
    mf = getattr(measures, measure['function'])
    cc = centers[lbl['center']] if ('center' in lbl.keys()) else None
    measurement = mf(lbl_mask, img_clean, cc)
    print(measure['function'],measurement)

1 liver
volume [957354]
surface_area [255486]
color_mean [134.92143658458627]
color_mean_dilated [106.40123660512828]
color_mean_eroded [131.5065935521058]
color_median [134.0]
color_median_dilated [110.0]
color_median_eroded [131.0]
color_std [18.96998865566971]
color_std_dilated [36.39823159943829]
color_std_eroded [15.039514243706465]
color_perc_1 [93.0]
color_perc_1_dilated [46.0]
color_perc_1_eroded [97.0]
color_perc_99 [190.0]
color_perc_99_dilated [181.0]
color_perc_99_eroded [177.0]
convex_volume [1533284.5]
radius_minimal_sphere [100.39669406502752]
eccentricity_meridional [0.2430539537752909]
eccentricity_equatorial [0.08963000738917838]


### eyes

In [20]:
id = 'Medaka_1115_20-1'
img_path = fpath + id + "/scaled_0.5_8bit_cropped_slices.tif"
img = tifffile.imread(img_path)

main_organ = 'eyes'
cfg_path = f"/home/ws/er5241/Repos/measuring-repo/measurement_configs/measurement/eye.yaml"

with open(cfg_path, "r") as stream:
    configs = yaml.safe_load(stream)

cleaning_config = configs['cleaning']
measuring_config = configs['measures']
centering_config = configs.get('centering', [])

msk_path = fpath + id + f"/{main_organ}_scaled_0.5_8bit_cropped_slices.tif"
msk = tifffile.imread(msk_path)
cleaner = DoubleStepCleaner(**cleaning_config)

msk_clean, roi = cleaner(msk)
img_clean = img[roi]

In [21]:
centers = {}
for centering in centering_config:
    centers[centering['name']] = Separator((msk_clean == centering['label_id']), centering['function'], centering['count'])

In [ ]:
for lbl in measuring_config:
  lbl_mask = (msk_clean == lbl['id'])
  print(lbl['id'], lbl['name'])
  for measure in lbl['measures']:
    mf = getattr(measures, measure['function'])
    cc = centers[lbl['center']] if ('center' in lbl.keys()) else None
    measurement = mf(lbl_mask, img_clean, cc)
    print(measure['function'],measurement)